# Tarea 2
## Laboratorio de Datos

# 1 Carga y transformación de Datos

In [ ]:
# librerías usadas
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import PIL
from skimage import io
from skimage.color import gray2rgb

# 1
# iniciar datos

"""
Funciones loaders 
- pil_loader        :   carga con pil
- skimage_loader    :   carga con skimage
- opencv_loader     :   carga con opencv
"""
def opencv_loader(path):
    bgr_image = cv2.imread(path)
    return bgr_image
def pil_loader(path):
    image_rgb =Image.open(path).convert("RGB")
    return image_rgb
def skimage_loader(path):
    image=io.imread(path)
    return gray2rgb(image)

div= np.iinfo('uint8').max # maximo valor de tipo uint8
transformers=transforms.Compose(
                [transforms.Resize([224,224]),
                #transforms.Normalize(mean=0,std = div, inplace = True),
                 transforms.RandomRotation(degrees=20),
                 transforms.RandomHorizontalFlip(),
                 transforms.ColorJitter(brightness=[1.2, 1.5]),
                 transforms.ToTensor() # ToTensor convierte a valores entre 0 y 1
                ])

# Usamos el loader por defecto de ImageFolder
# Deja las imagenes con 3 capas
from torchvision.datasets.folder import default_loader
    
data_train1=datasets.DatasetFolder(root='chest_xray/train',loader=default_loader,transform=transformers, extensions='jpeg')
data_test1=datasets.DatasetFolder(root='chest_xray/test',loader=default_loader,transform=transformers, extensions='jpeg')

In [ ]:
"""
To Do : 


Perfilamiento de tiempo de cómputo 


"""

In [ ]:
# 2

classes_train1 = data_train1.targets
classes_test1 = data_test1.targets

pneumonia_train=int(sum(classes_train1))
normal_train=len(classes_train1)-pneumonia_train


pneumonia_test=int(sum(classes_test1))
normal_test=len(classes_test1)-pneumonia_test

labels = 'Normal', 'Pneumonia'
sizes_train = [normal_train, pneumonia_train]

sizes_test = [normal_test, pneumonia_test]

fig, axs = plt.subplots(1,2)
axs[0].pie(sizes_train,labels=labels,explode=(0,0.1),autopct='%1.1f%%')
axs[1].pie(sizes_test,labels=labels,explode =(0,0.1),autopct='%1.1f%%')

axs[0].set_title('Train set')
axs[1].set_title('Test set')


plt.show()
plt.savefig('distribucionTrainTest.png')

In [ ]:
# 3

from sklearn.model_selection import train_test_split
from torch.utils.data import Subset

# split
train_idx, val_idx = train_test_split(list(range(len(data_train1))),test_size=0.2)
#data_train = Subset(data_train1, train_idx)
#data_val   = Subset(data_train1,val_idx)

class ReplicarMuestreoDePrueba(torch.utils.data.Sampler):
    
    def __init__(self,etiquetas_prueba, indices_val, etiquetas_val):
        self.indices_val      = indices_val
        #self.etiquetas_val    = etiquetas_val
        self.prob_pneumonia   = sum(etiquetas_prueba)/len(etiquetas_prueba)
        self.prob_normal      = 1-self.prob_pneumonia
        self.prob_vector      = [ int((etiquetas_val[i]==1 )*self.prob_pneumonia+
                                 (etiquetas_val[i]==0)*self.prob_normal)
                                for i in range(len(etiquetas_prueba))
                                ]
    def __iter__(self):
        return iter(np.random.choice(self.indices_val,p=self.prob_vector))
    
etiquetas_prueba = data_test1.targets
# indices_val = val_idx
etiquetas_val = [data_train1.targets[i] for i in val_idx ] 

a=ReplicarMuestreoDePrueba(etiquetas_prueba,val_idx, etiquetas_val)

In [ ]:
# 4
from torch.utils.data.sampler import RandomSampler, SubsetRandomSampler

data_train = DataLoader(data_train1,sampler=SubsetRandomSampler(train_idx))
data_val   = DataLoader(data_train1,sampler=ReplicarMuestreoDePrueba(etiquetas_prueba,val_idx, etiquetas_val))
data_test  = DataLoader(data_test1,sampler=RandomSampler(data_test1))

# P2

In [ ]:
#1 
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(2020)


class DWSepConv2d(nn.Module):
    
    def __init__(self,in_channels, out_channels, kernel_size,padding,bias=True,stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels,kernel_size,padding=padding,stride=stride,bias=bias)
        self.conv2 = nn.Conv2d(out_channels, out_channels,kernel_size=1,padding=padding,bias=bias,stride=stride)
    def forward(self,xb):
        xb = F.relu(self.conv1(xb.float()))
        xb = F.relu(self.conv2(xb)) 
        return xb

In [ ]:
#2

class VGG16DWSep(nn.Module):
    
    def __init__(self,in_channels):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels,64,kernel_size=3,padding=1,stride=1)
        self.conv2 = nn.Conv2d(64,64,kernel_size=3, padding=1, stride=1)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2,stride=2) # verificar los tamaños
        self.dwconv3 = DWSepConv2d(64,128,kernel_size=3,padding=1,stride=1)
        self.dwconv4 = DWSepConv2d(128,128,kernel_size=3,padding=1,stride=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2,stride=2)
        self.dwconv5 = DWSepConv2d(128,256,kernel_size=3,padding=1,stride=1)
        self.batchnorm1 = nn.BatchNorm2d(256)
        self.dwconv6 = DWSepConv2d(256,256,kernel_size=3,padding=1,stride=1)
        self.batchnorm2 = nn.BatchNorm2d(256)
        self.dwconv7 = DWSepConv2d(256,256,kernel_size=3,padding=1,stride=1)
        self.maxpool3 = nn.MaxPool2d(kernel_size=2,stride=2)
        self.dwconv8 = DWSepConv2d(256,512,kernel_size=3,padding=1,stride=1)
        self.batchnorm3 = nn.BatchNorm2d(512)
        self.dwconv9 = DWSepConv2d(512,512,kernel_size=3,padding=1,stride=1)
        self.batchnorm4 = nn.BatchNorm2d(512)
        self.dwconv10 = DWSepConv2d(512,512,kernel_size=3,padding=1,stride=1)
        self.maxpool4 = nn.MaxPool2d(kernel_size=2,stride=2)
        self.flatten1 = nn.Flatten()
        self.lin1 = nn.Linear(512,1024)
        self.drop1 = nn.Dropout(.7)
        self.lin2 = nn.Linear(1024,512)
        self.drop2 = nn.Dropout(.5)
        self.lin3 = nn.Linear(512,2)
    
    def forward(self,xb):
        xb = xb.view(-1,3,224,224).float()
        xb = F.relu(self.conv1(xb))
        xb = F.relu(self.conv2(xb))
        xb = F.relu(self.maxpool1(xb))
        xb = F.relu(self.dwconv3(xb))
        xb = F.relu(self.dwconv4(xb))
        xb = F.relu(self.maxpool2(xb))
        xb = F.relu(self.dwconv5(xb))
        xb = F.relu(self.batchnorm1(xb))
        xb = F.relu(self.dwconv6(xb))
        xb = F.relu(self.batchnorm2(xb))
        xb = F.relu(self.dwconv7(xb))
        xb = F.relu(self.maxpool3(xb))
        xb = F.relu(self.dwconv8(xb))
        xb = F.relu(self.batchnorm3(xb))
        xb = F.relu(self.dwconv9(xb))
        xb = F.relu(self.mbatchnorm4(xb))
        xb = F.relu(self.dwconv10(xb))
        xb = F.relu(self.flatten1(xb))
        xb = F.relu(self.lin1(xb))
        xb = F.relu(self.drop1(xb))
        xb = F.relu(self.lin2(xb))
        xb = F.relu(self.drop2(xb))
        xb = F.relu(self.lin3(xb))
        
        
        

In [ ]:
#3
import os
try:
    from urllib.request import URLopener
except ImportError:
    from urllib import URLopener

# Download VGG-16 weights from PyTorch
vgg_url = 'https://download.pytorch.org/models/vgg16_bn-6c64b313.pth'
if not os.path.isfile('./vgg16_bn-6c64b313.pth'):
    weights = URLopener().retrieve(vgg_url, './vgg16_bn-6c64b313.pth')

vgg16_weights = torch.load('./vgg16_bn-6c64b313.pth')

# Descargamos la red vgg16
vgg16 = torchvision.models.vgg16(pretrained=True, progress=True)


from torchsummary import summary
summary(vgg16.cuda(),(3,224,244))




# Se deben obtener las dos primeras capas de convolucion de la red VGG16
for i in range(3):
  print(vgg16.features[i])


vgg16.state_dict

# Se quiere la capa 0 y la 2, que son las dos primeras convolucionales:
pesos_dict = {
    'conv1' : vgg16.features[0],
    'conv2' : vgg16.features[2]
}


net = VGG16DWSep(in_channels = 3)

#intento de traspaso de pesos:
net.conv1 = pesos_dict['conv1']
net.conv2 = pesos_dict['conv2']


# Congelamos los pesos de la red vgg16
for param in vgg16.parameters():
  param.requires_grad = False

# Congelamos los pesos de las dos primeras capas convoluvionales de la red 
# (capas que se transfirieron):
net.conv1.requires_grad_ = False
net.conv2.requires_grad_ = False


net.cuda()


# P3

In [ ]:
#1

# librerías usadas
import torch
from torchvision import models
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# iniciar datos
def load(path):
    image_rgb =Image.open(path).convert("RGB")
    return image_rgb



# transformaciones compose
transformers=transforms.Compose([transforms.Resize([229,229]),
                transforms.CenterCrop(229),
                 transforms.ToTensor(),
                 transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225], inplace = True)
                ])

In [ ]:
# 2
from keras.applications.imagenet_utils import decode_predictions
inception_v3_net = models.inception_v3(pretrained=True)

In [ ]:
inception_v3_net.eval()
filename = 'dog.jpg'
input_image = Image.open(filename)
input_tensor = transformers(input_image).float()
input_batch = input_tensor.unsqueeze(0)
inception_v3_net(input_batch)